# ***Ana Lorena Jiménez Preciado***
## Uso de API Banxico

### **Descripción General del Código**

Este código está diseñado para interactuar con el Sistema de Información Económica (SIE) de Banxico, permitiendo el acceso y la descarga de datos económicos de manera programática. A continuación, se detallan los aspectos clave:

- **API del SIE de Banxico**: La API proporciona acceso a una amplia gama de series estadísticas económicas, que son esenciales para análisis económicos y financieros.
- **Uso de la API**: Para utilizar esta API, es necesario registrarse y obtener un token de acceso. Este token es fundamental para autenticar y autorizar las peticiones HTTP que se realizan a la API.
- **Descarga de datos**: El código automatiza la descarga de datos desde el SIE mediante solicitudes específicas, que incluyen el identificador de la serie de datos, la fecha de inicio y la fecha de fin.
- **Protección de acceso**: Es importante mantener el token de acceso de manera segura y no exponerlo públicamente para evitar usos no autorizados.

Para obtener el token necesario y comenzar a utilizar esta API, visite el siguiente enlace: [Banxico SIE API](https://www.banxico.org.mx/SieAPIRest/service/v1/;jsessionid=fbc95bfa19ade25ba51c6317a93f).




### **Importaciones necesarias**

1. **numpy (np)**:
   - Utilizado para realizar operaciones matemáticas y trabajar con arrays.
   - En este código, se usa para manejar operaciones como cambios porcentuales y la función `where`.
   ```python
   import numpy as np

2. **pandas (pd)**:
   - Utilizada para manipular y analizar datos estructurados.
   - Se emplea para crear y manipular DataFrames, realizar cambios de formato en las fechas, y calcular cambios porcentuales.
   - Pandas es fundamental en la manipulación de datos debido a su capacidad para leer, filtrar, y transformar datos de manera eficiente.
   ```python
   import pandas as pd

3. **requests**:
   - Usada para realizar peticiones HTTP.
   - Necesaria para descargar datos de la API de Banxico utilizando su endpoint REST.
   - Permite el acceso a datos remotos de manera programática, esencial para la integración de datos en tiempo real o actualizados regularmente.
   ```python
   import requests

4. **plotly.graph_objects (go)**:
   - Usada para crear gráficos interactivos.
   - Reemplaza a matplotlib para realizar visualizaciones dinámicas y más interactivas de los datos.
   - Plotly es especialmente útil para crear visualizaciones que requieren interactividad del usuario, como tooltips, zoom y actualizaciones dinámicas.
   ```python
   import plotly.graph_objects as go


5. **datetime**:
   - Utilizada para manipular fechas y horas.
   - Aunque se importa en el código original, en la versión optimizada y documentada no se utiliza directamente si todas las operaciones de fecha se manejan con pandas, que ofrece funcionalidades más robustas para trabajar con series temporales.
   ```python
   from datetime import datetime



In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import plotly.graph_objects as go

In [2]:
def descarga_bmx_series(series_dict, fechainicio, fechafin):
    # Token de acceso a la API de Banxico
    token = ''
    headers = {'Bmx-Token': token}
    all_data = []  # Lista para almacenar los DataFrames de cada serie

    # Itera sobre el diccionario de series proporcionado
    for serie, nombre in series_dict.items():
        # Construye la URL para la consulta API usando la serie y las fechas
        url = f'https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie}/datos/{fechainicio}/{fechafin}/'
        response = requests.get(url, headers=headers)  # Realiza la petición HTTP

        # Verifica si la respuesta es exitosa
        if response.status_code != 200:
            print(f'Error en la consulta para la serie {nombre}, código {response.status_code}')
            continue

        raw_data = response.json()  # Obtiene los datos en formato JSON

        # Verifica la estructura del JSON y si contiene datos
        if 'bmx' in raw_data and 'series' in raw_data['bmx'] and len(raw_data['bmx']['series']) > 0:
            serie_data = raw_data['bmx']['series'][0]

            if 'datos' in serie_data and len(serie_data['datos']) > 0:
                data = serie_data['datos']
                df = pd.DataFrame(data)
                df['dato'] = df['dato'].replace('N/E', np.nan).astype(float)  # Convierte los datos a float y maneja los no disponibles
                df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)  # Convierte la columna de fecha al formato datetime
                df.set_index('fecha', inplace=True)  # Establece la columna de fecha como índice del DataFrame
                df.rename(columns={'dato': nombre}, inplace=True)  # Renombra la columna de datos con el nombre de la serie
                all_data.append(df)  # Agrega el DataFrame a la lista all_data
            else:
                print(f"No se encontraron datos para la serie {nombre}")
        else:
            print(f"Estructura inesperada en la respuesta para la serie {nombre}")

    # Concatena todos los DataFrames en uno solo si se descargaron datos
    if all_data:
        df_final = pd.concat(all_data, axis=1)
        return df_final
    else:
        print("No se descargaron datos.")
        return None

## **Relevancia del INPC y Componentes de Inflación**

### ¿Qué es el INPC?
El **Índice Nacional de Precios al Consumidor (INPC)** es un indicador económico diseñado para medir la variación de los precios de una canasta de bienes y servicios representativa del consumo de los hogares en México. Este índice es vital para evaluar la inflación, que refleja el aumento generalizado de precios en la economía. El INPC se calcula utilizando la fórmula de Laspeyres, y es reconocido internacionalmente por su metodología avanzada y congruente con las mejores prácticas internacionales.

### ¿Cómo se construye el INPC?
El INPC se construye con base en una metodología que incluye:
- Una canasta de 299 conceptos genéricos ponderados según la Encuesta Nacional de Gastos de los Hogares (ENGASTO).
- Representación de todas las localidades urbanas del país, asegurando que cada estado esté representado por al menos una ciudad.
- Inclusión de diversos tipos de bienes y servicios consumidos por familias urbanas y rurales.

### Inflación Subyacente
La **inflación subyacente** es una medida que excluye elementos volátiles y productos con precios afectados por factores externos, como alimentos sin procesar y energéticos. Este indicador proporciona una visión más estable de la inflación y es crucial para la formulación de políticas monetarias y fiscales. Ayuda a medir la inflación de fondo sin ruidos temporales.

### Inflación No Subyacente
La **inflación no subyacente** incluye productos cuyos precios son más susceptibles a variaciones debido a condiciones climáticas o políticas gubernamentales específicas, como alimentos frescos y tarifas autorizadas por el gobierno.

### Utilidad del INPC
El INPC es fundamental para:
- Ajustar créditos fiscales y valores nominales.
- Determinar el valor de la Unidad de Inversión (UDI) y la Unidad de Medida y Actualización (UMA).
- Negociaciones contractuales y ajustes de precios en contratos privados y públicos.
- Funcionar como deflactor del Sistema de Cuentas Nacionales, ayudando a mantener la estabilidad del poder adquisitivo de la moneda.

### Fuentes
Para más información, visitar [INEGI - INPC](https://www.inegi.org.mx/programas/inpc/2018/PreguntasF/).



In [3]:
# Define las series de datos y sus códigos correspondientes
series = {
    'SP1': 'INPC',
    'SP74625': 'Subyacente',
    'SP74630': 'No_Subyacente'
}

# Define el rango de fechas de interés
fechainicio = '2010-01-01'
fechafin = pd.Timestamp.today().strftime('%Y-%m-%d')

# Llama a la función y asigna el resultado a df_final
df_final = descarga_bmx_series(series, fechainicio, fechafin)

# Verifica si se descargaron datos y muestra los primeros registros
if df_final is not None:
    print("Datos descargados exitosamente ✅ :")
    print(df_final.tail())  # Muestra los últimos registros para confirmar la descarga
else:
    print("No se pudo descargar ningún dato.")

Datos descargados exitosamente ✅ :
               INPC  Subyacente  No_Subyacente
fecha                                         
2024-09-01  136.080  135.433359     137.879203
2024-10-01  136.828  135.812502     139.885808
2024-11-01  137.424  135.875585     142.307614
2024-12-01  137.949  136.567452     142.257207
2025-01-01  138.343  137.134007     142.059801


In [4]:
df_final

,INPC,Subyacente,No_Subyacente
fecha,,,
2010-01-01,72.552046,76.019222,62.745041
2010-02-01,72.971671,76.333280,63.440733
2010-03-01,73.489725,76.601139,64.615797
2010-04-01,73.255565,76.683689,63.545574
2010-05-01,72.793978,76.862500,61.393449
...,...,...,...
2024-09-01,136.080000,135.433359,137.879203
2024-10-01,136.828000,135.812502,139.885808
2024-11-01,137.424000,135.875585,142.307614


## Cálculo de la inflación a partir del índice

Cálculo de Inflaciones Anualizadas

Inflación General:

$$ \text{Inflación}_t = \left(\frac{\text{INPC}_t - \text{INPC}{t-12}}{\text{INPC}{t-12}}\right) \times 100 $$

Inflación Subyacente:

$$ \text{Core}_t = \left(\frac{\text{Subyacente}_t - \text{Subyacente}{t-12}}{\text{Subyacente}{t-12}}\right) \times 100 $$

Inflación No Subyacente:
$$ \text{Inflación No Subyacente}_t = \left(\frac{\text{No Subyacente}_t - \text{No Subyacente}{t-12}}{\text{No Subyacente}{t-12}}\right) \times 100 $$

### Análisis respecto al Objetivo de Inflación

Desviación del Objetivo:
$$ \text{Inflación vs Target}_t = \text{Inflación}_t - 3 $$

Indicador de Cumplimiento del Rango Objetivo:

$$ \text{Inflación en Target}_t = \begin{cases}
1 & \text{si } 2 \leq \text{Inflación}_t \leq 4 \\
0 & \text{en otro caso}
\end{cases} $$

Objetivo de Inflación:

$$ \text{Target Inflación} = 3% $$

## Filtrado Temporal

Selección de Muestra:
$$ t \geq \text{2012-01-01} $$

Donde:

$t$ representa el período de tiempo actual
$t-12$ representa el mismo período del año anterior
Todas las inflaciones están expresadas en términos porcentuales


In [5]:
df_final['Inflacion'] = df_final['INPC'].pct_change(12) * 100
df_final['Core'] = df_final['Subyacente'].pct_change(12) * 100
df_final['Inflacion_No_Subyacente'] = df_final['No_Subyacente'].pct_change(12) * 100
df_final['Inflacion_vs_target'] = df_final['Inflacion'] - 3
df_final['inflation_in_target'] = np.where(df_final['Inflacion'].between(2, 4), 1, 0)
df_final['Target_Inflacion'] = 3

df = df_final[df_final.index >= '2012-01-01']

In [6]:
df

,INPC,Subyacente,No_Subyacente,Inflacion,Core,Inflacion_No_Subyacente,Inflacion_vs_target,inflation_in_target,Target_Inflacion
fecha,,,,,,,,,
2012-01-01,78.343049,81.133182,70.345435,4.046773,3.343693,6.375294,1.046773,0,3
2012-02-01,78.502314,81.478176,69.985427,3.868634,3.367104,5.530868,0.868634,1,3
2012-03-01,78.547389,81.677294,69.600459,3.729278,3.311350,5.123184,0.729278,1,3
2012-04-01,78.300980,81.800316,68.322809,3.412079,3.388984,3.485602,0.412079,1,3
2012-05-01,78.053819,82.017578,66.781612,3.851229,3.479343,5.148802,0.851229,1,3
...,...,...,...,...,...,...,...,...,...
2024-09-01,136.080000,135.433359,137.879203,4.580387,3.912806,6.504091,1.580387,0,3
2024-10-01,136.828000,135.812502,139.885808,4.761540,3.803007,7.683170,1.761540,0,3
2024-11-01,137.424000,135.875585,142.307614,4.548671,3.577505,7.599834,1.548671,0,3


In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['Inflacion'], mode='lines+markers', name='Inflacion'))
fig.add_trace(go.Scatter(x=df.index, y=df['Core'], mode='lines+markers', name='Core (Inflación Subyacente)'))
fig.add_trace(go.Scatter(x=df.index, y=df['Target_Inflacion'], mode='lines+markers', name='Objetivo de inflación'))
fig.add_trace(go.Scatter(x=df.index, y=df['Inflacion_No_Subyacente'], mode='lines+markers', name='Inflación No Subyacente'))

fig.update_layout(
    title='Evolución de la Inflación, Inflación Subyacente, no subyacente y target de inflación (Desde 2012)',
    xaxis_title='Fecha',
    yaxis_title='Inflación anualizada (%)',
    legend_title='Legend',
    template='plotly_white'
)

fig.show()

In [8]:
# Guardar el DataFrame df_final en un archivo CSV
df.to_csv('inflacion.csv')